In [ ]:
import pandas as pd
from jellyfish import jaro_winkler

In [ ]:
survey_2016 = "/Users/chris/Code/TempeSurveyReport/tempe_survey_data_2016.csv"

In [ ]:
df16 = pd.read_csv(survey_2016)

In [ ]:
df_describe = df16.describe()

In [ ]:
df16

In [ ]:
for _id in df16["ID"]:
    print(_id)

In [ ]:
def normalize(col):
    min_val = col["min"]
    norm = 1/(col["max"]-min_val)
    return col.apply(lambda x: (x-min_val)*norm)
df_describe = df_describe.drop("count",axis=0)

In [ ]:
df_norm = df_describe.apply(normalize,axis=0)
row_indexes = df_norm.index

In [ ]:
def get_distance(col1,col2):
    return sum(abs(df_norm[col1][row]-df_norm[col2][row]) for row in row_indexes)

In [ ]:
from itertools import combinations
def get_top_k_features(k=5):
    cols_distances = {}
    key_pairs = combinations(df_norm.keys(),2)
    for cols in key_pairs:
        key = tuple(sorted(cols))
        if key not in cols_distances:
            cols_distances[key] = get_distance(key[0], key[1])
    return sorted(cols_distances.items(), key=lambda x: x[1])[:k]

In [ ]:
get_top_k_features()

In [ ]:
def get_full_col_name():
    pass

def autocomplete_col_names():
    pass

In [ ]:
import re

df16 = pd.read_csv(survey_2016)
full_col_names_file = "/Users/chris/Code/TempeSurveyReport/full_questions_2016.txt"
full_col_names = []
with open(full_col_names_file, "r") as f:
    full_col_names = f.read().split("\n")

full_name_question_map = {}
special_questions = {
    "9": " The following adequately support my work-related needs:",
    "10": " The following programs/services adequately support my needs:",
}

for text in full_col_names:
    period_loc = text.find(".")
    question_number = text[:period_loc]
    if question_number:
        stripped_number = re.sub("[^0-9]", "", question_number)
        if stripped_number in special_questions:
            special_text = (
                "Q"
                + question_number
                + "."
                + special_questions[stripped_number]
                + text[period_loc + 1 :]
            )
            full_name_question_map["Q" + question_number] = special_text
        else:
            full_name_question_map["Q" + question_number] = "Q" + text

In [ ]:
print(full_name_question_map)

In [ ]:
rename_dict = {}
for key in df16.keys():
    rename_dict[key] = key.split(" ")[0]
df16 = df16.rename(columns=rename_dict)
section_keys = df16.keys()

In [ ]:
df16 = df16.rename(columns=full_name_question_map)

In [ ]:
df16

In [ ]:
key_map = {}
current_keys = set(df16.keys())-set(["ID"])
for col in df16.keys():
    top_sim = 0
    best_key = ""
    for match_col in full_col_names:
        sim = jaro_winkler(col,match_col)
        if sim > top_sim:
            top_sim = sim
            best_key = match_col
    key_map[col] = best_key

In [ ]:
key_map

In [ ]:
df16 = df16.set_index("ID")

In [ ]:
df16.describe()